In [3]:
using NotebookSIAMFANL
using SIAMFANLEquations
using LinearAlgebra
using SIAMFANLEquations.TestProblems
using PyPlot

## Section 1.10 Solvers for Chapter 1

Contents for Section 1.10

[Overview](#Overview)

[nsolsc.jl](#nsolsc.jl)


### Overview

We provide two solvers for this chapter. __nsolsc.jl__ is a scalar Newton code. Its calling sequence and the tuple it returns are very similar to all the codes from this book. All our solvers return a tuple with the solution, the history of the iteration, flags for success or failure, and (optionally) the entire history of the solution. 

The solution hisory for scalar equations is small and returning it is the default. In the later chapters on systems of equations, we do not return the solution history by default and discourage your asking for it. The solution history might take a lot of space to store and also, especially in Julia, have a severe penalty for allocations.

### nsolsc.jl

__nsolsc.jl__ is the scalar Newton solver. We will begin, as we will in all the software sections, by looking at the documentation in the code.

In [4]:
? nsolsc

search: nsolsc



nsolsc(f,x, fp=difffp; rtol=1.e-6, atol=1.e-12, maxit=10,         solver="newton", sham=1, armmax=10, resdec=.1,         armfix=false, printerr=true, keepsolhist=true)

Newton's method for scalar equations. Has most of the features a code for systems of equations needs.

Input:

f: function

x: initial iterate

fp: derivative. If your derivative function is fp, you give me its name. For example fp=foobar tells me that foobar is your function for the derivative. The default is a forward difference Jacobian that I provide.

Options:

rtol, atol: real and absolute error tolerances

maxit: upper bound on number of nonlinear iterations

solver:

Your choices are "newton"(default), "secant", or "chord". However,  you have sham at your disposal only if you chose newton. "chord" will keep using the initial derivative until the iterate converges, uses the iteration budget, or the line search fails. It is not the same as sham=Inf, which is smarter.

If you use secant and your initial iterate is poor, you have made a mistake. I will help you by driving the line search with a finite difference derivative.

sham:

This is the Shamanskii method. If sham=1, you have Newton. The iteration updates the derivative every sham iterations. The covergence rate has local q-order sham+1 if you only count iteratons where you update the derivative. You need not provide your own derivative function to use this option. sham=Inf is chord only if chord is converging well.

armmax: upper bound on stepsize reductions in linesearch

resdec: target value for residual reduction. 

The default value is .1. In the old MATLAB codes it was .5. I only turn Shamanskii on if the residuals are decreasing rapidly, at least a factor of resdec, and the line search is quiescent. If you want to eliminate resdec from the method ( you don't ) then set resdec = 1.0 and you will never hear from it again.  

armfix:

The default is a parabolic line search (ie false). Set to true and the stepsize will be fixed at .5. Don't do this unless you are doing experiments for research.

printerr:

I print a helpful message when the solver fails. To supress that message set printerr to false.

keepsolhist:

Set this to true to get the history of the iteration in the output tuple. This is on by default for scalar equations and off for systems. Only turn it on if you have use for the data, which can get REALLY LARGE.

Output:

A tuple (solution, functionval, history, stats, idid, solhist) where history is the vector of residual norms (|f(x)|) for the iteration and stats is a tuple of the history of (ifun, ijac, iarm), the number of functions/derivatives/steplength reductions at each iteration.

I do not count the function values for a finite-difference derivative because they count toward a Jacobian evaluation. I do count them for the secant method model.

idid=true if the iteration succeeded and false if not.

solhist:

This is the entire history of the iteration if you've set keepsolhist=true


#### Input


Let's begin with the calling sequence for the solver.

```julia
nsolsc(
    f,
    x,
    fp = difffp;
    rtol = 1.e-6,
    atol = 1.e-12,
    maxit = 10,
    solver = "newton",
    sham = 1,
    armmax = 5,
    resdec = .1,
    armfix = false,
    keepsolhist = true,
)
```

The arguments before the semicolon are required. We are solving $f(x) = 0$ and the solver needs $f$ and the initial iterate $x$. The arguments after the semicolon are __keyword arguments__, usually refered to as __kwargs__, which is not a German cheese product. The semicolon is __very important__. Do not leave it out when using kwargs. The
good news about kwargs is that you may use any of them without worrying about the others, which will take their default values. So

```julia
nsolout0 = nsolsc(atan, 1.0)

nsolout1 = nsolsc(atan, 1.0; solver = secant)

nsolout2=nsolsc(atan, 3.0; sham=2, resdec=.5)
```
are all correct.


You have seen many of the kwargs before. The realtive and absolute error tolerances, the solver, the parameters
for the Shamanskii method and line search should be familiar. The new things are __resedec__, __armfix__, and __keepsolshist__. For example, the derivative is updated every __sham__ iterations. Newton's method is sham=1.

The default for derivative evaluation is a forward difference derivative. That is an internal function __difffp__.
If you have an analytic derivative, say __fpanal.jl__, then set fp=fpanal and the solver will use your derivative.

The documentation explains these parameters. We have mentioned the solution history before. Pleaes leave __keepsolhist__ alone unless there's a good reason to change it. It is set to true for scalar codes and false for
the solvers in the following chapters.

__resdec__ is how we manage Shamanskii iterations. In this scalar code, it is used for some examples and to prepare you for its more serious use in the codes for systems of equations. In __nsolsc.jl__ the default solver is Newton's method (so sham=1). Newton with sham=2 is the Shamanskii method with a derivative update every two iterations. 
__But__ we safeguard the skipping of the update by doing the update anyhow if (1) the line search fails on the first attempt (ie with step length = 1) or (2) the residual decrease is more than __resdec__. If you want to eliminate the second of these, set resdec = 1. The only exception to the first criterion is the chord method. If you set solver='chord' then you will get chord. That's in there for research and a few internal tests of the code. 
If you set sham=Inf, then you'll get the chord method with derivative updates when a step length of 1 fails to produce sufficient decrease or the reduction in residuals is not enough.

We also compute a difference derivative in the secant method if the line search kicks in. As I said in the comment lines near that part of the code ...

```Julia

    # If you like the secant or sham=large methods, I will do a
    # difference Jacobian anyhow if the line search kicks in.
    # You will thank me for this.
    # Even if you don't thank me, I will do it anyhow.
```



#### Output

The output of all the solvers is a tuple. This is a data structure in Julia that can pack differnet structures (including more tuples) in one thing. It's a good way to manage complex output.

__nsolsc.jl__ 